# 第8回 その1: 類似度ベースのクラスタリング：ユークリッド距離と正規分布
ここでは，類似度を用いてクラスタリングを行う方法として，  
クラス平均とのユークリッド距離を用いる方法と，  
正規分布（多変量正規分布）を用いる方法とを比較します。  


## ステップ1: データの作成
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

まず，二種類のクラスに属するデータを生成します。  
これらのデータは，それぞれ異なる正規分布に基づいて生成されています。

In [ ]:
# 乱数のシード設定
np.random.seed(0)

# 分布の平均値ベクトルと分散共分散行列を設定
# (以下の値はシミュレーションデータの平均と分散共分散の設定)
mean1 = np.array([2, 4])
mean2 = np.array([8, 6])
cov1 = np.array([[5, 4], [4, 5]])
cov2 = np.array([[0.5, 0], [0, 0.5]])

# 分布に基づいてデータを200サンプル生成
class1 = np.random.multivariate_normal(mean1, cov1, size=200)
class2 = np.random.multivariate_normal(mean2, cov2, size=200)
# 各クラスの平均値ベクトルと分散共分散行列を計算
mean1 = np.mean(class1, axis=0)
mean2 = np.mean(class2, axis=0)
cov1 = np.cov(class1.T)
cov2 = np.cov(class2.T)
print('class 1')
print('mean = ')
print(mean1)
print('covariance matrix = ')
print(cov1)
print('\nclass 2')
print('mean = ')
print(mean2)
print('covariance matrix = ')
print(cov2)

# データと平均値をプロット
plt.figure(figsize=(8,8))
plt.scatter(class1[:,0], class1[:,1], color='deepskyblue', label='class 1')
plt.scatter(mean1[0], mean1[1], color='b', label='mean of class 1', marker='x', s=100)
plt.scatter(class2[:,0], class2[:,1], color='salmon', label='class 2')
plt.scatter(mean2[0], mean2[1], color='r', label='mean of a class 2', marker='x', s=100)
plt.legend()
plt.show()

各データは２次元の多変量ベクトルです。  
<font color='deepskyblue'>水色</font>の点はクラス１に属するデータで，$\times$ 印はクラス１の平均値ベクトルです。  
<font color='salmon'>赤色</font>の点はクラス２に属するデータで，$\times$ 印はクラス２の平均値ベクトルです。  

次に，あるサンプル sample $= [5, 8]$ を作成し，<font color='green'>緑色の$\times$印</font>でプロットします。

In [ ]:
sample = np.array([5, 8])

# データと平均値をプロット
plt.figure(figsize=(8,8))
plt.scatter(class1[:,0], class1[:,1], color='deepskyblue', label='class 1')
plt.scatter(mean1[0], mean1[1], color='b', label='mean of class 1', marker='x', s=100)
plt.scatter(class2[:,0], class2[:,1], color='salmon', label='class 2')
plt.scatter(mean2[0], mean2[1], color='r', label='mean of a class 2', marker='x', s=100)
plt.scatter(sample[0], sample[1], color='g', label='sample', marker='x', s=100)
plt.legend()
plt.show()

さて，このサンプルはクラス1とクラス2，どちらに分類されるべきでしょうか？  
（この場合，緑のサンプル以外は属しているクラス情報（ラベル）があるので，**教師ありクラスタリング**ということになります。）

## ステップ2: 平均値ベクトルとのユークリッド距離  
クラス1とクラス2のどちらがクラスに近いか，平均値とのユークリッド距離を比較してみます。

In [ ]:
def euclidean_distance(x, y):
  '''
      ユークリッド距離を計算して出力する
      x, y: 距離を測りたい2個の多次元ベクトル
  '''
  dist = np.sum((x-y)**2)
  dist = np.sqrt(dist)
  return dist


print('class 1: %f' % (euclidean_distance(mean1, sample)))
print('class 2: %f' % (euclidean_distance(mean2, sample)))

ユークリッド距離の値は，小さい程そのクラスに近い，つまり類似しているという意味です。  
クラス2の方が平均値との距離が短かいので，**クラス2**に分類すべきという結果になります。

## ステップ3: 多変量正規分布関数を使った類似度計算
では，正規分布関数を使って類似度を計算した場合はどうでしょうか？

以下は正規分布関数の値を計算する関数です。

In [ ]:
def normal_distribution(x, mean, cov):
  '''
      正規分布の確率密度関数の値を計算する
      x: 計算したいデータ
      mean: 平均値ベクトル
      cov: 分散共分散行列
  '''
  # 次元数
  D = np.size(x)
  # 分散共分散行列の逆行列
  inv_cov = np.linalg.inv(cov)
  # 分散共分散行列の行列式
  det_cov = np.linalg.det(cov)
  
  sub = x - mean
  # sub は1行k列のベクトルなので，これをk行1列のベクトルに変換する
  sub = np.reshape(sub, [-1,1])

  tmp = np.dot(sub.T, inv_cov)
  tmp = np.dot(tmp, sub)
  tmp = np.exp(-1.0 * tmp / 2)
  res = tmp / (np.sqrt((2*np.pi)**D * det_cov))

  return res[0,0]

定義した関数を使ってクラス1，クラス2それぞれに関する類似度を計算します。

In [ ]:
print('class 1: ' + str(normal_distribution(sample, mean1, cov1)))
print('class 2: ' + str(normal_distribution(sample, mean2, cov2)))

正規分布関数の値は，高い程そのクラスに近いことを意味します。  
結果を見ると，クラス1の方が類似度が高いため，**クラス1**に分類すべきという結果になります。  

分布の図を見ると，確かにクラス2よりクラス1の方が分類先としては正しそうです。

ユークリッド距離は，あくまで平均値との距離しかみておらず，データの分布の仕方，つまり分散共分散を考慮していません。  
従って，データ間の距離や類似度を測る時，背後にあるデータの分布情報が分かっている場合は，ユークリッド距離よりも分布の関数を使う方が良いと言えます。